In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import neurokit2 as nk
import xarray as xr
from bibliotheque import *
from params import *

## PARAMS

In [3]:
srate = down_srate
srate

250

In [4]:
save = True

In [5]:
ecg_interesting_metrics = ['HRV_MeanNN', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_pNN50', 'HRV_pNN20', 'HRV_LF', 'HRV_HF', 'HRV_LFHF', 'HRV_S', 'HRV_CSI', 'HRV_CVI']
ppg_interesting_metrics = ['mean amplitude', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_pNN50', 'HRV_pNN20', 'HRV_LF', 'HRV_HF', 'HRV_LFHF', 'HRV_S', 'HRV_CSI', 'HRV_CVI']
eda_interesting_metrics = ['nb_peaks', 'mean_height', 'EDA_Symp', 'EDA_SympN']

## TOOLS

In [6]:
def eeg_to_metrics(eeg_sig, srate=srate):
    metrics = nk.eeg_power(eeg_sig, sampling_rate=srate)
    return metrics
    
def ecg_to_metrics(ecg_sig, interesting_metrics, srate=srate, get_fci_signal=False):
    peaks, info_ecg = nk.ecg_peaks(ecg_sig, sampling_rate=srate,method='neurokit', correct_artifacts=True)
    R_peaks = info_ecg['ECG_R_Peaks'] # get R time points
    metrics = nk.hrv(R_peaks, sampling_rate=srate)
    
    if get_fci_signal:
        diff_R_peaks = np.diff(R_peaks) 
        x = time_vector(ecg, srate)
        xp = R_peaks[1::]/srate
        fp = diff_R_peaks
        interpolated_hrv = np.interp(x, xp, fp, left=None, right=None, period=None) / srate
        fci = 60 / interpolated_hrv
        
        return metrics[interesting_metrics], fci
    else:
        return metrics[interesting_metrics]
    
def gsr_homemade_metrics(info, dict_symp):   
    nb_peaks = info['SCR_Peaks'].size
    mean_height = np.mean(info['SCR_Height'])
    mean_amp = np.mean(info['SCR_Amplitude'])
    mean_risetime = np.mean(info['SCR_RiseTime'])
    mean_recov_time = np.mean(info['SCR_RecoveryTime'])
    
    data = [nb_peaks, mean_height, mean_amp, mean_risetime, mean_recov_time, dict_symp['EDA_Symp'], dict_symp['EDA_SympN']]
    df = pd.Series(data=data, index = ['nb_peaks', 'mean_height', 'mean_amp', 'mean_risetime', 'mean_recov_time','EDA_Symp','EDA_SympN'])
    return df

def eda_to_metrics(eda_sig, interesting_metrics, srate=srate):
    __ , info = nk.eda_process(eda_sig, sampling_rate=srate, method='neurokit')
    dict_symp = nk.eda_sympathetic(eda_sig, sampling_rate = srate)
    metrics = gsr_homemade_metrics(info, dict_symp)
    return metrics[interesting_metrics]
    
def ppg_amplitude(sig, srate, show=False):
    idx_maxs = signal.find_peaks(sig, distance = srate/2)[0]
    idx_mins = signal.find_peaks(-sig, distance = None)[0]
 
    amplitudes = []
    val_creux = []
    val_sommets = []
    indices_creux = []
    indices_sommets = []
    for idx_sommet in idx_maxs:
        cond = idx_mins < idx_sommet
        if not sum(cond) == 0:
            idx_creux = idx_mins[cond][-1]
            value_creux = sig[idx_creux]
            value_sommet = sig[idx_sommet]
            
            
            val_creux.append(value_creux)
            val_sommets.append(value_sommet)
            indices_sommets.append(idx_sommet)
            indices_creux.append(idx_creux)
        
            amplitude = value_sommet - value_creux
            amplitudes.append(amplitude)
    # print(np.mean(amplitudes))
    

    # df = pd.DataFrame(np.array([np.arange(1,len(val_sommets)+1 , 1), indices_sommets , indices_creux, val_sommets, val_creux, amplitudes]).T, columns = ['cycle','idx_max','idx_min','val_max','val_min','amplitude'])

    if show:
        plt.figure()
        plt.plot(sig)
        plt.plot(indices_sommets, val_sommets, 'x')
        plt.plot(indices_creux, val_creux, 'o')
        plt.show()
        
    return np.mean(amplitudes)

def ppg_to_metrics(ppg_sig, interesting_metrics, srate=srate): 
    peaks = nk.ppg_findpeaks(ppg_sig, sampling_rate=srate, method='elgendi', show=False)
    metrics = nk.hrv(peaks, sampling_rate=srate, show=False)
    amp = ppg_amplitude(ppg_sig, srate)
    metrics.insert(0, 'mean amplitude', amp)
    return metrics[interesting_metrics]

## LOAD DATA

In [7]:
da = xr.load_dataarray('../data_preprocessed/da_cleaned.nc').sel(cleaning = 'clean') # sel already cleaned data

In [8]:
da

<xarray.DataArray (participant: 10, room: 9, dtype: 5, time: 22501)>
array([[[[-1.98951966e-13,  1.08895065e+01, -3.37755401e+00, ...,
           3.41684774e+01, -8.50098734e+00, -4.61852778e-14],
         [ 8.52651283e-14,  3.05753678e+01,  1.67270879e+01, ...,
           8.46988846e-01, -8.02165826e+01,  7.10542736e-14],
         [-3.99481870e+00, -3.23349538e+00, -2.40230460e+00, ...,
           9.78090929e+00,  5.59578507e+00,  3.78464397e+00],
         [ 1.35440469e+02,  1.35449113e+02,  1.35457694e+02, ...,
           8.01139701e+01,  8.01139069e+01,  8.01138556e+01],
         [ 2.75827749e+01,  2.98107142e+01,  3.18742657e+01, ...,
          -1.12543721e+01, -1.29490417e+01, -1.47835626e+01]],

        [[ 1.13686838e-13,  3.35867958e+01,  1.12906401e+01, ...,
           3.92670513e+01, -5.89494907e+01, -9.94759830e-14],
         [-9.94759830e-14, -7.16583846e+01,  7.11922899e+01, ...,
          -4.63795439e+02, -4.28517025e+02, -1.42108547e-13],
         [ 9.72328585e-01,  2.62541341e+00,  4.91714400e+00, ...,
           4.97008135e+00,  6.03783369e+00,  7.11076450e+00],
         [ 1.07809219e+02,  1.07891931e+02,  1.07973620e+02, ...,
           1.40340527e+02,  1.40341398e+02,  1.40342099e+02],
         [-2.69893018e-02,  2.01497270e+00,  3.89058503e+00, ...,
...
          -2.99876903e+01, -2.80821745e+01,  2.13162821e-13],
         [ 0.00000000e+00,  4.37559506e+01,  6.46471717e+01, ...,
           6.24751970e+01,  4.83304741e+01, -8.70414851e-14],
         [-9.42113903e+00, -1.08326964e+01, -1.20440094e+01, ...,
           4.81577218e+00,  8.09066262e-01, -3.07567600e+00],
         [ 5.06149847e+02,  5.06173040e+02,  5.06197113e+02, ...,
           5.35893370e+02,  5.35893465e+02,  5.35893535e+02],
         [-2.01922380e+01, -1.92856702e+01, -1.84234983e+01, ...,
          -5.42107512e+00, -7.30528286e+00, -9.34294077e+00]],

        [[ 1.42108547e-14,  2.30040988e+01, -2.68752993e+01, ...,
           2.16938503e+01,  1.72684956e+01, -7.10542736e-14],
         [-4.26325641e-14,  2.76928688e+01,  9.47472674e+01, ...,
           8.87992007e+01,  4.54760851e+01, -8.52651283e-14],
         [ 9.96178214e+00,  7.35815367e+00,  5.42694538e+00, ...,
           1.07208759e+01,  1.20805289e+01,  1.37301393e+01],
         [ 5.33934997e+02,  5.33839116e+02,  5.33741007e+02, ...,
           4.71550497e+02,  4.71550328e+02,  4.71550195e+02],
         [ 3.47926981e+01,  3.70874212e+01,  3.91445050e+01, ...,
          -3.90873088e+00, -5.50253597e+00, -7.21033397e+00]]]])
Coordinates:
  * participant  (participant) object 'P01PPILNI' 'P03PBABCO' ... 'P17ETRPA'
  * room         (room) object 'Baseline' 'Take-off' ... 'End of the world'
  * dtype        (dtype) object 'EEGL' 'EEGR' 'ECG' 'EDA' 'PPG'
    cleaning     <U5 'clean'
  * time         (time) float64 0.0 0.004 0.008 0.012 ... 89.99 89.99 90.0 90.0

## DATA TO METRICS

In [14]:
eeg_concat = []
ecg_concat = []
eda_concat = []
ppg_concat = []

for participant in participants:
    print(participant)
    for room in rooms:
        print(room)
        for dtype in dtypes:
            # print(dtype)
            sig = da.loc[participant, room, dtype, :].values
            
            # plt.figure()
            # plt.plot(sig[0:2500])
            # plt.title(dtype)
            # plt.show()
            
            if not np.mean(sig) == 0: # don't compute metrics if no signal
                if dtype in ['EEGL','EEGR']:
                    metrics = eeg_to_metrics(eeg_sig=sig)
                    metrics.insert(0, 'chan', dtype)
                    metrics.insert(0, 'room', room)
                    metrics.insert(0, 'participant', participant)
                    eeg_concat.append(metrics)
                elif dtype == 'ECG':
                    metrics = ecg_to_metrics(ecg_sig=sig, interesting_metrics = ecg_interesting_metrics)
                    metrics.insert(0, 'room', room)
                    metrics.insert(0, 'participant', participant)
                    ecg_concat.append(metrics)
                elif dtype == 'EDA':
                    metrics = eda_to_metrics(eda_sig=sig, interesting_metrics = eda_interesting_metrics)
                    metrics['room'] = room
                    metrics['participant'] = participant
                    eda_concat.append(metrics)
                elif dtype == 'PPG':
                    metrics = ppg_to_metrics(ppg_sig=sig, interesting_metrics = ppg_interesting_metrics)
                    metrics.insert(0, 'room', room)
                    metrics.insert(0, 'participant', participant)
                    ppg_concat.append(metrics)     
            
eeg_metrics = pd.concat(eeg_concat).drop(columns = ['Channel'])
ecg_metrics = pd.concat(ecg_concat)
eda_metrics = pd.concat(eda_concat, axis = 1).T
ppg_metrics = pd.concat(ppg_concat)

P01PPILNI
Baseline


/home/valentin/.virtualenvs/py38/lib/python3.8/site-packages/neurokit2/hrv/hrv_nonlinear.py:441: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
/home/valentin/.virtualenvs/py38/lib/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/valentin/.virtualenvs/py38/lib/python3.8/site-packages/neurokit2/hrv/hrv_nonlinear.py:441: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(


Take-off


/home/valentin/.virtualenvs/py38/lib/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '


Savannah


...

/home/valentin/.virtualenvs/py38/lib/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '


Sea


/home/valentin/.virtualenvs/py38/lib/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '


End of the world


/home/valentin/.virtualenvs/py38/lib/python3.8/site-packages/scipy/signal/spectral.py:1964: UserWarning: nperseg = 128 is greater than input length  = 113, using nperseg = 113
  warnings.warn('nperseg = {0:d} is greater than input length '


In [15]:
if save:
    eeg_metrics.to_excel('../metrics/eeg_metrics.xlsx')
    ecg_metrics.to_excel('../metrics/ecg_metrics.xlsx')
    eda_metrics.to_excel('../metrics/eda_metrics.xlsx')
    ppg_metrics.to_excel('../metrics/ppg_metrics.xlsx')

* To extract the tonic component of the EDA signals (SCL), a 10th-order low-pass finite impulse response filter with a cut-off frequency of 0.0004 Hz was applied. 
* The remaining signal (raw signal miuns tonic component) was used to compute NS.SCRs. The SCL index was computed using the mean of the tonic EDA over the 2-min period. 
* NS.SCRs were obtained manually for each minute and then averaged over the 2-min period. It is important to note that for defining a non-negligible occurence of NS.SCRs, a minimum change in conductance needs to be considered. A recommanded threshold value of 0.05 µS was used. In addition, when a second response occured before completion of the prior response, the two responses were counted as two positive NS.SCRs even though they overlapped.